In [ ]:
from langchain.document_loaders.rspace import RSpaceLoader
import os

from dotenv import load_dotenv

load_dotenv()

GHA_TOKEN = os.getenv('GHA')
os.getenv('OPENAI_API_KEY')

In [50]:
rspace_rfp="SD1932247"
walks= "NB182183"

In [51]:
loader = RSpaceLoader(global_id=walks)
docs=loader.load()
docs[0].to_json()
len(docs)


/Users/richard.c.adams/.local/share/virtualenvs/jupyter_notebooks-h594mpzp/lib/python3.10/site-packages/rspace_client/eln/field_content.py:21: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  self.soup = BeautifulSoup(self.html, "html.parser")


20

In [52]:
len(docs)
summaries = [d.page_content[0:100] for d in  docs]
print('\n----\n'.join(summaries))


<h2>Carn Bhac<h2/>Date walked
10/09/2023
mountains climbed
Can Bhac
Route
Walk highlands from Invera
----
<h2>West Ben Lawers range<h2/>Date walked
30/11/2019
mountains climbed
Meall Corranaich, Meall a Cho
----
<h2>Cruach ardrain<h2/>Date walked
22/04/2023
mountains climbed
Cruach ardrain
Benn tulaichean 
Rout
----
<h2>Loch nan Eun Munros<h2/>Date walked
09/07/2022
mountains climbed
Beinn iutharn mor
An Socach 
Ca
----
<h2>Carn a chlaim<h2/>Date walked

mountains climbed

Route

Time taken (hrs)

Notes



 
 

----
<h2>ben vorlich (Arrochar)<h2/>Date walked
09/11/2019
mountains climbed
Ben Vorlich, Arrochar, WH ro
----
<h2>Ime narnain<h2/>Date walked
21/09/2019
mountains climbed

Route

Time taken (hrs)

Notes



 

----
<h2>An Casteal<h2/>Date walked
06/03/2019
mountains climbed
an casteal, beinn achabair, beinn a chro
----
<h2>Glen lyon horseshoe <h2/>Date walked
24/02/2019
mountains climbed
Glen lyon 4 munros 
Route
Stan
----
<h2>Ben Wyvis<h2/>Date walked
17/02/2019
mountains climb

In [59]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores.utils import filter_complex_metadata
filtered = filter_complex_metadata(docs)

In [61]:
c = CharacterTextSplitter(chunk_size=100, chunk_overlap=20)
r = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=40)
all_chunks = r.split_documents(filtered)
all_chunks[0]
len(all_chunks)

    

20

In [62]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.vectorstores.utils import filter_complex_metadata


In [63]:
filtered = filter_complex_metadata(all_chunks)
filtered[0].metadata
len(filtered)
filtered[2]

Document(page_content='<h2>Cruach ardrain<h2/>Date walked\n22/04/2023\nmountains climbed\nCruach ardrain\nBenn tulaichean\xa0\nRoute\nStandard why route from inverlochlarig\nTime taken (hrs)\n4\nNotes\nSunny, solo, cold on top. Great views of southern Highlands.', metadata={'source': 'rspace: Cruach ardrain-SD1835518'})

In [64]:
v_store = Chroma.from_documents(documents=filtered, embedding=OpenAIEmbeddings())


In [57]:
results = v_store.similarity_search(query = "alignment", k=4, score_threshold=0.8)
results

[Document(page_content='Implementation Approach', metadata={'source': 'rspace: rob1-SD1932247'}),
 Document(page_content='Implementation Approach', metadata={'source': 'rspace: rob1-SD1932247'}),
 Document(page_content='that perspective to ensure that RSpace interoperates with tools and services at both ends of the workflow.', metadata={'source': 'rspace: rob1-SD1932247'}),
 Document(page_content='that perspective to ensure that RSpace interoperates with tools and services at both ends of the workflow.', metadata={'source': 'rspace: rob1-SD1932247'})]

In [28]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo


In [30]:
results = sqretriever.get_relevant_documents("created by richarda23", verbose=True)
results


NameError: name 'sqretriever' is not defined

In [71]:
from langchain.chains import RetrievalQA
from langchain.chat_models  import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.9, verbose=True)
retriever = v_store.as_retriever( search_kwargs={'k':4})
vector_results = retriever.get_relevant_documents(query="what is my longest walk")

vector_results

[Document(page_content='<h2>Carn a chlaim<h2/>Date walked\n\nmountains climbed\n\nRoute\n\nTime taken (hrs)\n\nNotes', metadata={'source': 'rspace: Carn a chlaim-SD488647'}),
 Document(page_content='<h2>Ime narnain<h2/>Date walked\n21/09/2019\nmountains climbed\n\nRoute\n\nTime taken (hrs)\n\nNotes', metadata={'source': 'rspace: Ime narnain-SD488651'}),
 Document(page_content='<h2>An Casteal<h2/>Date walked\n06/03/2019\nmountains climbed\nan casteal, beinn achabair, beinn a chroin\nRoute\nAs in 100 best routes in scottish mountains\nTime taken (hrs)\n8.5\nNotes', metadata={'source': 'rspace: An Casteal-SD488640'}),
 Document(page_content='<h2>Loch nan Eun Munros<h2/>Date walked\n09/07/2022\nmountains climbed\nBeinn iutharn mor\nAn Socach\xa0\nCarn an Righ\nRoute\nOvernight camp at Loch nan Eun. Walk in from the Spittal of Glenshee, set up camp, then baim in the evening.\xa0\nNext day an socach via sw scree slopes and then Carn an Righ.\xa0\n\xa0\nTime taken (hrs)\n11\nNotes\n\n\n\n\n\n

In [76]:
qa_chain = RetrievalQA.from_chain_type(llm,chain_type='stuff',retriever=retriever,verbose=True )

result = qa_chain({"query": "what walks has the weather been good on"})
result



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'what walks has the weather been good on',
 'result': "The weather has been good on the following walks:\n\n- Carn a Chlaim (date walked not provided)\n- Beinn Dearg (15/09/2018)\n- Ime Narnain (date walked not provided)\n- Carn Bhac (10/09/2023)\n\nPlease note that the dates for Ime Narnain are not provided, so we don't have information about the weather on that specific day."}

In [35]:
result = qa_chain({"query": "what tools does RSpace integrate with  "})
result



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'what tools does RSpace integrate with  ',
 'result': 'RSpace integrates with several tools and systems. This includes the eCAT sample tracking system, third-party sample tracking, freezer management, colony management, instrument data capture systems, lab supply & procurement systems like Quartzy and others. It also integrates with popular commercial data sources including Box, Dropbox, Google Drive, MS OneDrive, Mendeley, and the eCAT sample tracking system, as well as archiving systems like DSpace and Arkivum. In addition, RSpace has the capability to connect with generic SQL and other databases. However, it currently does not integrate with project management tools but this functionality is part of their future plans.'}

In [ ]:
result

In [ ]:
#v_store.delete_collection()